<a href="https://colab.research.google.com/github/gichurejoy/Machine-Learning/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk -q

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install datasets -q

     |████████████████████████████████| 441 kB 10.3 MB/s 
     |████████████████████████████████| 212 kB 51.3 MB/s 
     |████████████████████████████████| 115 kB 43.4 MB/s 
     |████████████████████████████████| 95 kB 5.4 MB/s 
     |████████████████████████████████| 163 kB 44.2 MB/s 
     |████████████████████████████████| 127 kB 48.3 MB/s 
     |████████████████████████████████| 115 kB 56.5 MB/s 


In [4]:
from datasets import load_dataset
dataset = load_dataset("tweet_eval", "emoji")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [7]:
train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]

test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

texts = dataset["validation"]["text"]
labels = dataset["validation"]["label"]

In [17]:

print(len(texts), len(labels))

5000 5000


In [18]:
import pandas as pd

df = pd.DataFrame()

In [19]:
df["texts"] = texts
df["labels"] = labels
df

,texts,labels
0,A little throwback with my favourite person @ ...,0
1,glam on @user yesterday for #kcon makeup using...,7
2,Democracy Plaza in the wake of a stunning outc...,11
3,Then &amp; Now. VILO @ Walt Disney Magic Kingdom,0
4,Who never... @ A Galaxy Far Far Away,2
...,...,...
4995,don't know what I'd do without you @ Keene Sta...,0
4996,Senior night with my little Bailey !! So proud...,3
4997,Real friends or labeled as family! #BrotherMan...,6
4998,It makes me so happy meet people wearing hats ...,3


In [20]:
df['labels'].value_counts()

0     1056
1      521
2      504
3      308
4      243
5      238
6      204
7      199
8      177
9      171
10     159
13     153
15     148
11     143
16     137
19     131
14     129
12     129
18     129
17     121
Name: labels, dtype: int64

In [21]:
df = df[df['labels'].isin([1, 2])]
df['labels'].value_counts()

1    521
2    504
Name: labels, dtype: int64

In [22]:
print(df['texts'].iloc[2])

#livinginparadise @ Ramada Plaza Beach Resort


In [23]:
df['texts']=df['texts'].str.replace("@","")
df['texts']=df['texts'].str.replace("#","")
df['texts']=df['texts'].str.replace("\d+","")
df['texts']=df['texts'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/us

In [25]:
import itertools

vocab=list(itertools.chain.from_iterable(df['texts']))
vocab=list(set(vocab))
len(vocab)

57

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

bow=CountVectorizer()
bow.fit(df['texts'])

CountVectorizer()

In [27]:
bow=bow.transform(df['texts'])
print(bow.shape)

(1025, 3675)


In [30]:
from sklearn.model_selection import train_test_split as ts

Xt,Xts,yt,yts=ts(bow,df['labels'])
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score as acc

knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(Xt,yt)
knn_pred=knn.predict(Xts)
print(acc(knn_pred,yts))

nb=MultinomialNB()
nb.fit(Xt,yt)
nb_pred=nb.predict(Xts)
print(acc(nb_pred,yts))

0.5836575875486382
0.6459143968871596


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 1))
tfidf.fit(df['texts'])
tfidf = tfidf.transform(df['texts'])
print(tfidf.shape)


(1025, 3675)


In [35]:
from sklearn.model_selection import train_test_split as ts

Xt,Xts,yt,yts=ts(tfidf,df['labels'])
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score as acc

knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(Xt,yt)
knn_pred=knn.predict(Xts)
print(acc(knn_pred,yts))

nb=MultinomialNB()
nb.fit(Xt,yt)
nb_pred=nb.predict(Xts)
print(acc(nb_pred,yts))

0.6731517509727627
0.7042801556420234
